<a href="https://colab.research.google.com/github/anshumanmudgal/E-Commerce/blob/main/Music_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import librosa

Music Recommender System Class


In [ ]:
class MusicRecommender:
    def __init__(self):
        self.features_df = None
        self.scaler = StandardScaler()
        self.trained = False
        self.n_mfcc = 20  # Increased number of MFCCs
        self.feature_names = None


 # Extracting features
    def extract_features(self, audio_file):

      """Extract audio features from a file"""
      try:
          # Load the audio file
          y, sr = librosa.load(audio_file, duration=30)

          features = {}

          # Extract MFCCs
          mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=self.n_mfcc)
          for i, mfcc in enumerate(mfccs):
              features[f'mfcc_{i+1}_mean'] = np.mean(mfcc)
              features[f'mfcc_{i+1}_var'] = np.var(mfcc)
              features[f'mfcc_{i+1}_max'] = np.max(mfcc)
              features[f'mfcc_{i+1}_min'] = np.min(mfcc)

          # Spectral features
          spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
          features['spectral_centroid_mean'] = np.mean(spectral_centroids)
          features['spectral_centroid_var'] = np.var(spectral_centroids)

          spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
          features['spectral_rolloff_mean'] = np.mean(spectral_rolloff)
          features['spectral_rolloff_var'] = np.var(spectral_rolloff)

          # Chroma features
          chroma = librosa.feature.chroma_stft(y=y, sr=sr)
          features['chroma_mean'] = np.mean(chroma)
          features['chroma_var'] = np.var(chroma)

          # Zero crossing rate
          zcr = librosa.feature.zero_crossing_rate(y)[0]
          features['zcr_mean'] = np.mean(zcr)
          features['zcr_var'] = np.var(zcr)

          # Tempo and beat features
          tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
          features['tempo'] = tempo

          # RMS energy
          rms = librosa.feature.rms(y=y)[0]
          features['rms_mean'] = np.mean(rms)
          features['rms_var'] = np.var(rms)

          if self.feature_names is None:
              self.feature_names = list(features.keys())

          # Ensure all features are present
          for name in self.feature_names:
              if name not in features:
                  features[name] = 0.0

          return features

      except Exception as e:
          print(f"Error extracting features from {audio_file}: {str(e)}")
          return None


          # Model Train
    def train(self, data_path):
      """Train the recommender system using the provided dataset"""
      try:
          print("Loading data...")
          # Load the dataset
          df = pd.read_csv(data_path)

          # Store feature names
          self.feature_names = list(df.columns)

          # Standardize features
          print("Processing features...")
          self.features_df = pd.DataFrame(
              self.scaler.fit_transform(df),
              columns=self.feature_names
          )

          self.trained = True
          print("Training completed.")

      except Exception as e:
          print(f"An error occurred during training: {str(e)}")
          raise

    def get_recommendations(self, input_features, n_recommendations=5):
      """Get recommendations based on input features"""
      try:
          if not self.trained:
              raise ValueError("Model not trained yet. Please train first.")

          # Convert features to DataFrame with correct column order
          input_df = pd.DataFrame([input_features])[self.feature_names]

          # Standardize input features
          input_scaled = self.scaler.transform(input_df)

          # Calculate similarities
          similarities = []
          for idx, row in self.features_df.iterrows():
              similarity = np.dot(input_scaled[0], row) / (
                  np.linalg.norm(input_scaled[0]) * np.linalg.norm(row)
              )
              similarities.append((idx, similarity))

          # Sort by similarity
          similarities.sort(key=lambda x: x[1], reverse=True)

          return similarities[:n_recommendations]

      except Exception as e:
          print(f"An error occurred while generating recommendations: {str(e)}")
          print(f"Debug info - Input features shape: {len(input_features)}, Expected features: {len(self.feature_names)}")
          print(f"Missing features: {set(self.feature_names) - set(input_features.keys())}")
          return []

    def main():
      try:
            # Initialize recommender
            recommender = MusicRecommender()

            print("Training the recommender...")
            # Train with your dataset
            recommender.train('/content/Training_set_songs.csv')

            print("Generating recommendations...")
            # Extract features from a test song
            test_song = '/content/let_her_go.mp3'
            test_features = recommender.extract_features(test_song)

            if test_features:
              # Get recommendations
              recommendations = recommender.get_recommendations(test_features)

              print("\nRecommended songs:")
              for idx, similarity in recommendations:
                  print(f"Song {idx}: Similarity = {similarity:.3f}")
            else:
                print("Could not process test song.")

      except Exception as e:
              print(f"An error occurred: {str(e)}")

    if __name__ == "__main__":
      main()


Training the recommender...
Loading data...
Processing features...
Training completed.
Generating recommendations...

Recommended songs:
Song 741: Similarity = 0.631
Song 8497: Similarity = 0.611
Song 8528: Similarity = 0.611
Song 4019: Similarity = 0.610
Song 4028: Similarity = 0.591


Model Building

Feature Extraction

Model Training

Recommendation

Example


Training the recommender...
An error occurred: 'MusicRecommender' object has no attribute 'train'
